# Plot individual images on the cluster

This notebook explores how to view geospatial data on the cluster without needing to transfer it

## Imports

In [ ]:
import os
import sys
from pathlib import Path
import datetime

import rasterio
from rasterio import plot
from rasterio.plot import show
import matplotlib.pyplot as plt
import shutil
import tempfile
import json
import random
import numpy as np
import pandas as pd
import geopandas as gpd
from pyproj import Proj, transform
from pyproj import CRS
from shapely.geometry import box
from shapely.geometry import shape
from shapely.geometry import MultiPoint
from shapely.geometry import Point
from shapely.geometry import Polygon
import xarray as xr

%matplotlib inline

In [ ]:
##This is just for data checking. Not actually used in processing sequence so far.
def GetClosestImage(img_dir, imageType, YYYY, DDD):
    '''
    returns path of image closest to day DDD of year YYYY in directory of images
    Currently looks for images with name YYYYDDD* (DDD is day-of-year(1-365))
    or Sentinel images starting with L1C* (Date format is YYYYMMDD, starting at position 19)
    TODO: expand imageType to Landsat
    '''
    
    imgList = []
    
    if imageType == 'TS':
        for img in os.listdir(img_dir):
            if img.startswith(str(YYYY)):
                imgList.append(int(img[4:7]))
    elif imageType == 'L1C':
        for img in os.listdir(img_dir):
            if img.startswith('L1C') and img[19:23]==str(YYYY):
                MM = int(img[23:25])
                DD = int(img[25:27])
                ymd = datetime.datetime(YYYY, MM, DD)
                doy=int(ymd.strftime('%j'))
                imgList.append(doy)
    
    closestDay = min(imgList, key=lambda x:abs(x-DDD))
    print('closest day is: {}'.format(str(YYYY)+str(closestDay)))
    closestMM = (datetime.datetime(YYYY, 1, 1) + datetime.timedelta(closestDay - 1)).month
    closestDD = (datetime.datetime(YYYY, 1, 1) + datetime.timedelta(closestDay - 1)).day
    
    for fname in os.listdir(img_dir):
        if imageType == 'TS':
            if str(YYYY)+str(closestDay) in fname:
                closestimg = os.path.join(img_dir,fname)
        elif imageType == 'L1C':
            if str(YYYY)+'{:02d}'.format(closestMM)+'{:02d}'.format(closestDD) in fname and fname.startswith('L1C') and 'angles' not in fname:
                closestimg = os.path.join(img_dir,fname)
 
    return closestimg

In [ ]:
### For visualizing index image (Process check 1)
def exploreBand(img, spec_index):

    print('plotting {} band of image {}'.format(img, spec_index))
    fig, axarr = plt.subplots(1, 3, figsize=(15,5))
    
    if img.endswith('.tif'):
        with rasterio.open(img) as src:
            TSamp = src.read()
    elif img.endswith('.nc'):
        with xr.open_dataset(img) as xrimg:
            xrcrs = xrimg.crs
            print(xrcrs)  ##epsg:32632
            #print(xrimg.variables)
            print(xrimg.dims)
            #print(xrimg.coords)
    
        ### Check that x and y values correspond to UTM coords
        print("Coord range is: y: {}-{}. x: {}-{}".format(
          xrimg[spec_index]["y"].values.min(), 
          xrimg[spec_index]["y"].values.max(),
          xrimg[spec_index]["x"].values.min(), 
          xrimg[spec_index]["x"].values.max()))
    
        ### Get single band (Opens as Dataset)
        xr_idx = xrimg[spec_index]
    
        ##Note outliers (while nodata = None)
        origHist = xr_idx.plot.hist(color="purple")
        #ax.imshow()
        
        xr_idx_clean = xr_idx.where(xr_idx < 10000)
        maskedHist = xr_idx_clean.plot.hist(color="purple")
        #print('The no data value is:', xrimg[spec_index].rio.nodata)
    
        TSamp = xr_idx_clean

    if img.endswith('.nc'):
        xr_idx.plot.hist(color="purple",  ax=axarr[0])
        axarr[0].set_title("original data")
        xr_idx_clean.plot.hist(color="purple", ax=axarr[1])
        axarr[1].set_title("masked data")
        TSamp.plot(x="x",y="y")
        plot.show(TSamp, ax=axarr[2])
        axarr[2].set_title('{} band'.format(spec_index))
        axarr[2].axis('off')
    
    elif img.endswith('.tif'):
        plot.show(TSamp)
        axarr[2].set_title('{} band'.format(spec_index))
        axarr[2].axis('off')
        
    return axarr

In [ ]:
'''
PARAMETERS: modify in Notebook_settings notebook, then run that notebook and this cell to update here
DO not modify this cell
'''

%store -r basicConfig
print("Basic Parameters: \n brdf_dir = {} \n gridCell = {} \n index_dir = {} \n out_dir = {}"
      .format(basicConfig['brdf_dir'],basicConfig['gridCell'],basicConfig['index_dir'],basicConfig['out_dir']))

%store -r SinglePlotParams
print("Plotting Parameters: \n plotYr = {} \n plotDay = {} \n Viewband = {} \n imageType = {}"
      .format(SinglePlotParams['plotYr'],SinglePlotParams['plotDay'],SinglePlotParams['Viewband'],SinglePlotParams['imageType']))

In [ ]:
if SinglePlotParams['imageType'] == 'TS':
    SampImg = GetClosestImage(basicConfig['index_dir'],'TS',SinglePlotParams['plotYr'],SinglePlotParams['plotDay'])
    exploreBand(SampImg, basicConfig['spec_index'])
else:
    SampImg = GetClosestImage(basicConfig['brdf_dir'],'L1C',SinglePlotParams['plotYr'],SinglePlotParams['plotDay'])
    exploreBand(SampImg, SinglePlotParams['Viewband'])

## Plot 3-band image

In [ ]:
def normalize(array):
    array_min, array_max = array.min(), array.max()
    return (array - array_min) / (array_max - array_min)

def gammacorr(band, gamma):
    return np.power(band, 1/gamma)

with xr.open_dataset(SampImg) as xrimg:
    red = xrimg['red'].where(xrimg['red'] != 65535)
    green = xrimg['green'].where(xrimg['green'] != 65535)
    blue = xrimg['blue'].where(xrimg['blue'] != 65535)
    #print (red.min(), red.max())
    

red_g=gammacorr(red, SinglePlotParams['gamma'])
blue_g=gammacorr(blue, SinglePlotParams['gamma'])
green_g=gammacorr(green, SinglePlotParams['gamma'])

red_norm_g = normalize(red_g)
green_norm_g = normalize(green_g)
blue_norm_g = normalize(blue_g)

rgb = np.dstack([red_norm_g, green_norm_g, blue_norm_g])
fig = plt.figure(figsize=(18,12))
plt.imshow(rgb);

In [ ]:
### Run to print output as html

outName = str(basicConfig['country']+'1b_SampleImagePlot_for_Cell'+str(basicConfig['gridCell'])+'_from_'+str(SinglePlotParams['plotYr'])+str(SinglePlotParams['plotDay']))
!jupyter nbconvert --output-dir='./Outputs' --to html --no-input --output=$outName 1b_ExploreData_OpenImage.ipynb